In [1]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize

## Lecture des datas

In [2]:
df = pd.read_csv("all_data.csv", sep=";")
print(f"{len(df)} tickers")

1247 tickers


## Exclusions

### Tabac / Armes controversées

In [3]:
df2 = df.loc[(df['CWEAP_TIE']!=1) & (df['TOB_PRODUCER']!=1),:]
print(f"{len(df2)} tickers")

1237 tickers


Elimination de 10 entreprises

### Controverses 0/1

In [4]:
columns_controverse = ["E_CONTROVERSY", "S_CONTROVERSY", 'G_CONTROVERSY']

In [5]:
# Controverse E
for col in columns_controverse:
    print(f"NB {col} : {len(df.loc[df[col]<2,:])}")

NB E_CONTROVERSY : 28
NB S_CONTROVERSY : 149
NB G_CONTROVERSY : 35


In [6]:
df3 = df2.loc[~df[columns_controverse].lt(2).any(axis=1), :]
print(f"Elimination {len(df2)-len(df3)}")
print(f"Reste {len(df3)}")

Elimination 175
Reste 1062


### Note ESG < BB

In [7]:
print(f"NB : {len(df.loc[df["IVA_COMPANY_RATING"].isin(['B','CCC']),:])}")
df4 = df3.loc[~df["IVA_COMPANY_RATING"].isin(['B','CCC']),:]
print(f"Elimination {len(df3)-len(df4)}")
print(f"Reste {len(df4)}")

NB : 17
Elimination 11
Reste 1051


### Restrictions ODD

In [8]:
# odds = ["07","12","13"]
odds = [x for x in df.columns if x.startswith("SDG")]
for odd in odds:
    print(f"NB {odd} : {len(df.loc[df[odd]<=-2,:])}")

df5 = df4.loc[~df4[odds].le(-2).any(axis=1), :]
print(f"Elimination {len(df4)-len(df5)}")
print(f"Reste {len(df5)}")

NB SDG_01_NET_ALIGNMENT_SCORE : 28
NB SDG_02_NET_ALIGNMENT_SCORE : 7
NB SDG_03_NET_ALIGNMENT_SCORE : 48
NB SDG_04_NET_ALIGNMENT_SCORE : 16
NB SDG_05_NET_ALIGNMENT_SCORE : 1
NB SDG_06_NET_ALIGNMENT_SCORE : 14
NB SDG_07_NET_ALIGNMENT_SCORE : 83
NB SDG_08_NET_ALIGNMENT_SCORE : 6
NB SDG_09_NET_ALIGNMENT_SCORE : 37
NB SDG_10_NET_ALIGNMENT_SCORE : 9
NB SDG_11_NET_ALIGNMENT_SCORE : 51
NB SDG_12_NET_ALIGNMENT_SCORE : 106
NB SDG_13_NET_ALIGNMENT_SCORE : 83
NB SDG_14_NET_ALIGNMENT_SCORE : 64
NB SDG_15_NET_ALIGNMENT_SCORE : 66
NB SDG_16_NET_ALIGNMENT_SCORE : 28
NB SDG_17_NET_ALIGNMENT_SCORE : 70
Elimination 106
Reste 945


In [41]:
df5['EST_EU_TAXONOMY_MAX_REV'] = df5['EST_EU_TAXONOMY_MAX_REV'].fillna(0)
df5['Annualized return 10Y'] = df5['Annualized return 10Y'].fillna(0)
df5['Annualized return 10Y'] = df5['Annualized return 10Y'].fillna(0)
df5['INVEST_DURABLE'] = df5[['SDG_07_NET_ALIGNMENT_SCORE', 'SDG_12_NET_ALIGNMENT_SCORE', 'SDG_13_NET_ALIGNMENT_SCORE']].max(axis=1)

C:\Users\bapdu\AppData\Local\Temp\ipykernel_5924\3610090802.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df5['EST_EU_TAXONOMY_MAX_REV'] = df5['EST_EU_TAXONOMY_MAX_REV'].fillna(0)
C:\Users\bapdu\AppData\Local\Temp\ipykernel_5924\3610090802.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df5['Annualized return 10Y'] = df5['Annualized return 10Y'].fillna(0)
C:\Users\bapdu\AppData\Local\Temp\ipykernel_5924\3610090802.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from

In [42]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize

df6 = df5.loc[(df5['EST_EU_TAXONOMY_MAX_REV']>0) & (df5['ITR']>0),:] #Si Pas Taxo >0 trop de lignes => trop long a faire tourner
# Nombre de tickers
n = len(df6)
print(n)
perf_moy = df6['Annualized return 20Y'].values
taxonomie = df6['EST_EU_TAXONOMY_MAX_REV'].values
itr = df6['ITR'].values
durable = df6['INVEST_DURABLE']

# Fonction objectif (on minimise donc on prend -Perf_moy)
def objectif(w):
    return -np.dot(w, perf_moy)  # On minimise la valeur négative pour maximiser

# Contrainte : Taxonomie pondérée ≥ 10
def contrainte_taxonomie(w):
    return np.dot(w, taxonomie)-10

# Contrainte : ITR < 2
def contrainte_itr(w):
    return -np.dot(w, itr)+2

# Contrainte : Alignement environnemental pondéré ≥ 90%
def contrainte_alignement(w):
    filtres_durables = (durable >= 2).astype(int)  # 1 si durable > 2, sinon 0

    # Calcul de la somme des poids des entreprises ayant un score durable > 2
    return np.dot(w, filtres_durables) - 0.90  # Doit être ≥ 0

# Contrainte : Somme des poids = 1
def contrainte_somme(w):
    return np.sum(w) - 1

# Définition des contraintes sous forme de dictionnaires
contraintes = [
    {'type': 'eq', 'fun': contrainte_somme},  # Somme des poids = 1
    {'type': 'ineq', 'fun': contrainte_taxonomie},  # Taxonomie pondérée ≥ 10
    {'type': 'ineq', 'fun': contrainte_itr},  # ITR < 2
    {'type': 'ineq', 'fun': contrainte_alignement}  # Alignement pondéré ≥ 90%
]

# Bornes (chaque poids doit être positif et max 0.1)
bornes = [(0, 0.05) for _ in range(n)]

# Initialisation des poids (égalité répartie)
w0 = np.ones(n) / n

# Résolution avec SLSQP
resultat = minimize(objectif, w0, method='trust-constr', bounds=bornes, constraints=contraintes)

# Vérification des résultats
if resultat.success:
    w_opt = resultat.x
    df6['Poids'] = w_opt
else:
    print("L'optimisation a échoué :", resultat.message)

298


c:\Users\bapdu\anaconda3\Lib\site-packages\scipy\optimize\_differentiable_functions.py:551: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  self.H.update(delta_x, delta_g)
c:\Users\bapdu\anaconda3\Lib\site-packages\scipy\optimize\_differentiable_functions.py:551: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  self.H.update(delta_x, delta_g)
c:\Users\bapdu\anaconda3\Lib\site-packages\scipy\optimize\_differentiable_functions.py:551: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  self.H.update(d

In [43]:
print(f"Return PTF : {(df6['Poids']*df6['Annualized return 20Y']).sum()}")
print(f"TAXO : {(df6['Poids']*df6['EST_EU_TAXONOMY_MAX_REV']).sum()}") 
print(f"ITR : {(df6['Poids']*df6['ITR']).sum()}") 
print(f"Poids : {(df6['Poids']).sum()}") 

Return PTF : 0.2216689633328055
TAXO : 13.347672260092363
ITR : 1.9997340984582839
Poids : 1.0000000000333835


In [45]:
# Repartition sectorielle
(df6.groupby('gics_sector_name')['Poids'].sum())*100

gics_sector_name
Communication Services     0.074787
Consumer Discretionary     0.156234
Consumer Staples           0.193599
Financials                 5.029306
Health Care                0.106256
Industrials               40.438462
Information Technology    45.790292
Materials                  2.480220
Real Estate                5.545684
Utilities                  0.185160
Name: Poids, dtype: float64

In [47]:
df6[df6['Poids']>0.001]

,ticker,name,isin,country,currency,gics_sector_name,industry_group,region_en,Weight in MSCI World,Annualized return 20Y,...,FOOTPRINT_SCOPE_1,FOOTPRINT_SCOPE_2,FOOTPRINT_SCOPE_3,INTENSITY_SCOPE_1,INTENSITY_SCOPE_2,INTENSITY_SCOPE_3,EV,SALES,INVEST_DURABLE,Poids
1,NVDA US Equity,NVIDIA Corp,US67066G1040,UNITED STATES,USD,Information Technology,Semiconductors,North America,0.046132,0.396705,...,0.018164,0.224788,34.721009,0.131528,1.627755,251.425343,3.359642e+06,109406.527100,6.5,0.049970
7,AVGO US Equity,Broadcom Inc,US11135F1012,UNITED STATES,USD,Information Technology,Semiconductors,North America,0.015473,0.317303,...,0.297687,0.942548,69.793000,1.179637,3.735015,276.567279,8.371146e+05,49815.326600,1.5,0.049915
22,CRM US Equity,Salesforce Inc,US79466L3024,UNITED STATES,USD,Information Technology,Software,North America,0.004466,0.256363,...,0.062587,4.381089,59.867483,0.111356,7.794915,106.517341,2.710248e+05,35920.855100,3.5,0.049937
26,ORCL US Equity,Oracle Corp,US68389X1054,UNITED STATES,USD,Information Technology,Software,North America,0.003932,0.154035,...,0.073908,5.687098,54.076477,0.192933,14.845876,141.163849,5.873725e+05,53059.784700,4.0,0.045292
33,NOW US Equity,ServiceNow Inc,US81762P1021,UNITED STATES,USD,Information Technology,Software,North America,0.003177,0.232904,...,0.058149,1.315634,23.429482,0.240423,5.439600,96.871189,1.772127e+05,10107.177600,3.0,0.049921
129,EQIX US Equity,Equinix Inc,US29444U7000,UNITED STATES,USD,Real Estate,REITS,North America,0.001323,0.191146,...,1.261112,113.508815,85.419244,3.492161,314.318534,236.535387,9.751630e+04,8304.314625,6.0,0.001549
152,TT US Equity,Trane Technologies PLC,IE00BK9ZQ967,IRELAND,USD,Industrials,Miscellaneous Manufactur,Europe,0.001207,0.175237,...,9.849333,6.006117,12936.253220,11.020468,6.720275,14474.439530,8.885787e+04,18727.062380,2.5,0.049718
162,SHW US Equity,Sherwin-Williams Co/The,US8243481061,UNITED STATES,USD,Materials,Chemicals,North America,0.001149,0.193083,...,17.540197,10.728259,576.696940,19.514693,11.935937,641.615601,1.050580e+05,22267.375650,4.0,0.001140
232,FTNT US Equity,Fortinet Inc,US34959E1091,UNITED STATES,USD,Information Technology,Computers,North America,0.000869,0.259116,...,0.102035,0.416515,44.521282,0.240806,0.982984,105.071170,5.520495e+04,5516.061720,2.0,0.049936
294,CMI US Equity,Cummins Inc,US2310211063,UNITED STATES,USD,Industrials,Auto Manufacturers,North America,0.000693,0.181106,...,25.086658,60.878282,4864.568548,8.929238,21.668757,1731.473850,4.984372e+04,33022.189200,4.0,0.049118
